**Download necessary files**

In [1]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\sansk\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

**Import the necessary libraries**

In [2]:
import numpy as np
import random
import string

**Import libraries for scraping - beautifulSoup**

In [3]:
import bs4 as bs
import urllib.request
import re

**Read the web link**

In [5]:
import ssl
gcontext = ssl.SSLContext()
raw_html = urllib.request.urlopen('https://en.wikipedia.org/wiki/Natural_language_processing', context=gcontext)
raw_html = raw_html.read()

**Convert the web link contents into a file**

In [6]:
article_html = bs.BeautifulSoup(raw_html, 'lxml')

**Find only the html text containing inside < p > tags**

In [7]:
article_paragraphs = article_html.find_all('p')

**Read all the text inside paragraphs**

In [8]:
article_text = ''

for para in article_paragraphs:
    article_text += para.text
    
article_text

'Natural language processing (NLP) is a subfield of linguistics, computer science, and artificial intelligence concerned with the interactions between computers and human language, in particular how to program computers to process and analyze large amounts of natural language data.  The goal is a computer capable of "understanding" the contents of documents, including the contextual nuances of the language within them. The technology can then accurately extract information and insights contained in the documents as well as categorize and organize the documents themselves.\nChallenges in natural language processing frequently involve speech recognition, natural language understanding, and natural language generation.\nNatural language processing has its roots in the 1950s. Already in 1950, Alan Turing published an article titled "Computing Machinery and Intelligence" which proposed what is now called the Turing test as a criterion of intelligence, though at the time that was not articul

**Split the sentences**

In [9]:
corpus = nltk.sent_tokenize(article_text)
corpus

['Natural language processing (NLP) is a subfield of linguistics, computer science, and artificial intelligence concerned with the interactions between computers and human language, in particular how to program computers to process and analyze large amounts of natural language data.',
 'The goal is a computer capable of "understanding" the contents of documents, including the contextual nuances of the language within them.',
 'The technology can then accurately extract information and insights contained in the documents as well as categorize and organize the documents themselves.',
 'Challenges in natural language processing frequently involve speech recognition, natural language understanding, and natural language generation.',
 'Natural language processing has its roots in the 1950s.',
 'Already in 1950, Alan Turing published an article titled "Computing Machinery and Intelligence" which proposed what is now called the Turing test as a criterion of intelligence, though at the time th

**Clean the text**

In [10]:
for i in range(len(corpus )):
    corpus [i] = corpus [i].lower()
    corpus [i] = re.sub(r'\W',' ',corpus [i])
    corpus [i] = re.sub(r'\s+',' ',corpus [i])

**Create a dictionary of word frequency**

In [11]:
wordfreq = {}
for sentence in corpus:
    tokens = nltk.word_tokenize(sentence)
    for token in tokens:
        if token not in wordfreq.keys():
            wordfreq[token] = 1
        else:
            wordfreq[token] += 1

**Filter the top 200 words**

In [12]:
import heapq
most_freq = heapq.nlargest(200, wordfreq, key=wordfreq.get)

**Compute IDF values**

In [13]:
word_idf_values = {}
for token in most_freq:
    doc_containing_word = 0
    for document in corpus:
        if token in nltk.word_tokenize(document):
            doc_containing_word += 1
    word_idf_values[token] = np.log(len(corpus)/(1 + doc_containing_word))

**Compute TF values**

In [14]:
word_tf_values = {}
for token in most_freq:
    sent_tf_vector = []
    for document in corpus:
        doc_freq = 0
        for word in nltk.word_tokenize(document):
            if token == word:
                  doc_freq += 1
        word_tf = doc_freq/len(nltk.word_tokenize(document))
        sent_tf_vector.append(word_tf)
    word_tf_values[token] = sent_tf_vector

**Compute TF-IDF values**

In [15]:
tfidf_values = []
for token in word_tf_values.keys():
    tfidf_sentences = []
    for tf_sentence in word_tf_values[token]:
        tf_idf_score = tf_sentence * word_idf_values[token]
        tfidf_sentences.append(tf_idf_score)
    tfidf_values.append(tfidf_sentences)

In [16]:
tf_idf_model = np.asarray(tfidf_values)

In [17]:
tf_idf_model = np.transpose(tf_idf_model)

In [18]:
tf_idf_model

array([[0.00893094, 0.01786188, 0.05798245, ..., 0.        , 0.        ,
        0.        ],
       [0.06634413, 0.0497581 , 0.        , ..., 0.15422278, 0.15422278,
        0.        ],
       [0.04749637, 0.        , 0.06852471, ..., 0.        , 0.        ,
        0.14721266],
       ...,
       [0.01514377, 0.03028754, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.02048863, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [19]:
import pandas as pd
pd.DataFrame(tf_idf_model, columns=most_freq)

,the,of,and,in,to,language,a,natural,nlp,processing,...,amounts,goal,capable,contents,including,contextual,nuances,within,them,technology
0,0.008931,0.017862,0.057982,0.020419,0.036562,0.061256,0.024002,0.053408,0.031379,0.029724,...,0.083043,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.066344,0.049758,0.000000,0.000000,0.000000,0.037921,0.044576,0.000000,0.000000,0.000000,...,0.000000,0.154223,0.154223,0.154223,0.154223,0.154223,0.154223,0.154223,0.154223,0.000000
2,0.047496,0.000000,0.068525,0.036197,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.147213
3,0.000000,0.000000,0.047111,0.049771,0.000000,0.149312,0.000000,0.195273,0.000000,0.072452,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.038701,0.000000,0.000000,0.088481,0.000000,0.088481,0.000000,0.115717,0.000000,0.128804,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,0.016586,0.008293,0.017947,0.018960,0.000000,0.000000,0.044576,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,0.043538,0.021769,0.047111,0.000000,0.000000,0.049771,0.058506,0.065091,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,0.021325,0.014217,0.015383,0.000000,0.014550,0.016252,0.038208,0.021254,0.049950,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,0.019350,0.019350,0.000000,0.000000,0.039608,0.044241,0.000000,0.057859,0.000000,0.064402,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,0.029026,0.014513,0.000000,0.066361,0.000000,0.066361,0.039004,0.043394,0.000000,0.096603,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
